In [2]:
###############################
# v3.0 수정사항
# 과거 df 불러오는 방식을 txt 저장 방식에서
# 내부 저장소를 이용하여 갱신하는 방식으로 변경

In [47]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from itertools import count
import pandas as pd
from datetime import datetime, timedelta
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np

# import glob
# from pyexcel.cookbook import merge_all_to_a_book

In [28]:
chrome_dir='C:/Yeajin/chromedriver_win32/chromedriver'
save_dir='C:/Yeajin/리얼데이터/마일스톤/crawling/'

pstock_url='http://www.pstock.co.kr/2005pstock/main.asp'
naver_url='https://finance.naver.com/item/main.nhn?code='
url_38='http://www.38.co.kr/html/fund/index.htm?o=nw&page='

df_dtype={'code':str}

In [29]:
# update search list

def update_search_list():
    todo_names=[]
    todo_codes=[]
    df=pd.read_csv('기업리스트.csv',encoding='utf-8',index_col=[0],dtype={'code':str})
    # old_list=df['name'].tolist()
    old_name_list=df['name'].tolist()
    old_code_list=df['code'].tolist()

    fin=False

    for p in count(1):
        driver.get(url_38+str(p))
        for i in range(1,21):
            new_yn=driver.find_element_by_xpath(f'/html/body/table[3]/tbody/tr/td/table[1]/tbody/tr/td[1]/table[4]/tbody/tr[2]/td/table/tbody/tr[{i}]/td[9]')
            if new_yn.text.strip()!='예정':
                todo_name=driver.find_element_by_xpath(f'/html/body/table[3]/tbody/tr/td/table[1]/tbody/tr/td[1]/table[4]/tbody/tr[2]/td/table/tbody/tr[{i}]/td[1]/a')
                #####++======================================
                if todo_name.text.strip() in old_name_list:   ## old_name_list?
                    fin=True
                    print('업데이트가 완료되었습니다.')
                    break
                else:
                    todo_names.append(todo_name.text.strip())
                    todo_name.send_keys(Keys.CONTROL+'\n')
                    driver.switch_to.window(driver.window_handles[1])
                    todo_code=driver.find_element_by_xpath('/html/body/table[3]/tbody/tr/td/table[1]/tbody/tr/td[1]/table[2]/tbody/tr[2]/td[4]').text.strip()
                    todo_codes.append(todo_code.strip())
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
        if fin==True:
            break

    mCols=[]        
    updated_df=pd.DataFrame(mCols)

    n_name_list=todo_names+old_name_list
    n_code_list=todo_codes+old_code_list

    updated_df['name']=n_name_list
    updated_df['code']=n_code_list

    updated_df.to_csv('기업리스트.csv',encoding='utf-8')
    
    return todo_names, todo_codes

In [30]:
# create updated search list

def create_pstock_updated_search_list():
    
    new_stocks_1=[]
    new_stocks_2=[]
    new_stocks=todo_names
    for index,new_name in enumerate(new_stocks):
        if '(유가' in new_name or '(구' in new_name:
            new_name=new_name.split('(')[0]
            new_stocks_1.append(new_name)
        elif '스팩' in new_name and '호' in new_name:
            new_name=new_name.split('스팩')
            new_stocks_2.append(new_name)
        else:
            new_stocks_1.append(new_name)
    new_stocksss=new_stocks_1+new_stocks_2
    
    return new_stocksss,new_stocks_1, new_stocks_2

def create_naver_updated_search_list():
    naver_name_list=todo_names
    naver_code_list=todo_codes
    print(todo_names,todo_codes)
    
    return naver_name_list, naver_code_list


In [31]:
# 검색 함수

def pstock_search():
    
#     new_stocksss=create_pstock_updated_search_list()[0]
#     new_stocks_1=create_pstock_updated_search_list()[1]
#     new_stocks_2=create_pstock_updated_search_list()[2]
    
    driver.get(pstock_url)
    p_search=driver.find_element_by_xpath('//*[@id="head_main"]')
    p_search.clear()
# ===============================================================
# new_stocks_1 : 기업명 // new_stocks_2 : '스팩' 포함 기업명
# ===============================================================

    if new_name in new_stocks_1:
        p_search.send_keys(new_name)
        driver.find_element_by_xpath('//*[@id="up_win"]/table[2]/tbody/tr[1]/td[2]/table/tbody/tr/td[3]/table/tbody/tr[2]/td/table/tbody/tr/td[4]/input').click()
# =============================================================================
# 분리해서 검색하고, [기업명,ㅁ호] 중 기업명을 검색하여 ㅁ호를 포함한 이름을 선택
# =============================================================================

    elif new_name in new_stocks_2:
        p_search.send_keys(new_name[0])    
        driver.find_element_by_xpath('//*[@id="up_win"]/table[2]/tbody/tr[1]/td[2]/table/tbody/tr/td[3]/table/tbody/tr[2]/td/table/tbody/tr/td[4]/input').click()
        html=driver.page_source
        i=2
        while True:
            name=driver.find_element_by_xpath(f'/html/body/center/table/tbody/tr[2]/td[2]/table[2]/tbody/tr[3]/td[2]/table[2]/tbody/tr[{i}]/td[1]/a').text
            if new_name[1] in name:
                company=driver.find_element_by_xpath(f'/html/body/center/table/tbody/tr[2]/td[2]/table[2]/tbody/tr[3]/td[2]/table[2]/tbody/tr[{i}]/td[1]/a').click()
                break
            else:
                i+=1

In [32]:
def naver_search():
    searched=[]
    driver.get(naver_url+str(code))
    n=driver.find_element_by_xpath('//*[@id="middle"]/div[1]/div[1]/h2/a').text
    searched.append(n)

In [33]:
# 파싱 함수
def pstock_parsing():
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    tbodys=soup.find_all('tbody')
    for i in range(len(tbodys)-1,0,-1):
        if soup.find_all('tbody')[i].find_all('tr')[-1].find_all('td')[0].text=='상장예정 주식수':
            circulation_stocks=soup.find_all('tbody')[i].find_all('tr')[-3].find_all('td')[1].text.strip()
            circulation_share=soup.find_all('tbody')[i].find_all('tr')[-3].find_all('td')[2].text.strip()
            new_listing_stocks=soup.find_all('tbody')[i].find_all('tr')[-1].find_all('td')[1].text.strip()
            new_listing_share=soup.find_all('tbody')[i].find_all('tr')[-1].find_all('td')[2].text.strip()
    
    for j in range(len(tbodys)):
        try:
            if soup.find_all('tbody')[j].find_all('tr')[6].find_all('td')[0].text=='확정공모가':
                offering_price=soup.find_all('tbody')[j].find_all('tr')[6].find_all('td')[1].text[:-5]
        except:
            continue
    
    circulation_price=int(re.sub('[,]','',circulation_stocks))*int(re.sub('[,]','',offering_price))
    tot_price=int(re.sub('[,]','',new_listing_stocks))*int(re.sub('[,]','',offering_price))
    
    circulation_stocks_list.append(circulation_stocks)
    circulation_share_list.append(circulation_share)
    circulation_price_list.append(circulation_price)
    new_listing_stocks_list.append(new_listing_stocks)
    new_listing_share_list.append(new_listing_share)
    tot_price_list.append(tot_price)

In [58]:
def naver_parsing():
    tmp_price_list=[]
    tmp_date_list=[]
    
    driver.get(naver_url+str(code))
    #==
#     print(driver.find_element_by_xpath('//*[@id="middle"]/div[1]/div[1]/h2/a').text)
    #==

    driver.find_element_by_xpath('//*[@id="content"]/ul/li[2]/a').click()
    iframes=driver.find_elements_by_tag_name('iframe')
    driver.switch_to.frame(iframes[-2])

    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    pages=soup.find('table',class_='Nnavi').find_all('td')

    end_num=0
    if len(pages)!=1:
        driver.find_element_by_class_name('pgRR').click()
        cell_nums=[3,4,5,6,7,11,12,13,14,15]
        try:
            for i in cell_nums:
                price=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{i}]/td[7]/span').text
                #date=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{i}]/td[1]/span').text
                prev_i=i
                tmp_price_list.append(price)
                #tmp_date_list.append(date)
        except:
            first_price=tmp_price_list[-1]
            first_date=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{prev_i}]/td[1]/span').text
            first_price_list.append(first_price)
#             print(first_price)
    else:
        cell_nums=[3,4,5,6,7,11,12,13,14,15]
        try:
            for i in cell_nums:
                price=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{i}]/td[7]/span').text
#                 date=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{i}]/td[1]/span').text
                prev_i=i
                tmp_price_list.append(price)
        except:
            first_price=tmp_price_list[-1]
            first_date=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{prev_i}]/td[1]/span').text
            first_price_list.append(first_price)
#             print(first_price)
    
    
    n_dates=[]
    date=datetime.strptime(first_date,'%Y.%m.%d')
    for d in range(3):
        date=date+timedelta(weeks=4)    
        n_date=date.strftime('%Y.%m.%d')
        n_dates.append(n_date)
#         print(n_date)
    
    return first_price_list, first_date_list

In [51]:
# update df
def update_df():
    mCols=[]
    p_df=pd.DataFrame(mCols) 
    
    p_df['기업명']=search_list
    p_df['유통가능주식수']=circulation_stocks_list
    p_df['유통가능지분율']=circulation_share_list
    p_df['유통가능금액']=circulation_price_list
    p_df['상장예정주식수']=new_listing_stocks_list
    p_df['상장예정지분율']=new_listing_share_list
    p_df['시가총액']=tot_price_list
    p_df['상장첫날거래량']=first_price_list
    
    today=datetime.today().strftime('%Y%m%d')
    
    origin_df=pd.read_csv('pstock_naver.csv',encoding='utf-8',index_col=[0])
    new_df=pd.concat([p_df,origin_df])
    
#     last_point_note=open('date_info.txt','r',encoding='utf-8')
#     last_point=last_point_note.read().rstrip('\n')
#     origin_df=pd.read_csv(save_dir+f'pstock_naver_{last_point}.csv',encoding='utf-8',
#                           index_col=[0])
    
#     new_df=pd.concat([p_df,origin_df])
    
#     today_note=open('date_info.txt','w',encoding='utf-8')
#     print(f'{today}',file=today_note)
#     today_note.close()
    
    new_df.to_csv('pstock_naver.csv',encoding='utf-8')
    new_df.to_csv(save_dir+f'pstock_naver_{today}.csv',encoding='utf-8')
    

In [36]:
# create failed note
def failed_note():    
    today=datetime.today().strftime('%y%m%d')
    failed_note=open(save_dir+f'failed_note_{today}.txt','w',encoding='utf-8')
    print(f'pstock\n 검색불가 : {pstock_search_failed} \n파싱불가 : {pstock_parsing_failed}', file=failed_note)
    print(f'\n naver \n 검색불가 : {naver_search_failed} \n파싱불가 : {naver_parsing_failed}', file=failed_note)

    failed_note.close()
    

In [54]:
search_list=[]

pstock_search_failed=[]
pstock_parsing_failed=[]

naver_search_failed=[]
naver_parsing_failed=[]

company_name_list=[]

circulation_stocks_list=[]
circulation_share_list=[]
circulation_price_list=[]

new_listing_stocks_list=[]
new_listing_share_list=[]
tot_price_list=[]

first_price_list=[]
first_date_list=[]

In [55]:
# ================================================================
# 크롤링 수행   -   업데이트 시 수행
# ================================================================

driver=webdriver.Chrome(chrome_dir)

todo_names,todo_codes=update_search_list()

new_stocksss,new_stocks_1,new_stocks_2=create_pstock_updated_search_list()

naver_name_list,naver_code_list=create_naver_updated_search_list()

# pstock

for new_name in new_stocksss:
    try:
        pstock_search()     # search 함수 실행
        name=driver.find_element_by_xpath('/html/body/center/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td[1]/table[6]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/b').text.strip()
        search_list.append(name)
    except:
        search_list.append(np.nan)
        pstock_search_failed.append(new_name)
        circulation_stocks_list.append(np.nan)
        circulation_share_list.append(np.nan)
        circulation_price_list.append(np.nan)
        new_listing_stocks_list.append(np.nan)
        new_listing_share_list.append(np.nan)
        tot_price_list.append(np.nan)
        continue
        
    try:
        pstock_parsing()
        company_name_list.append(new_name)
    except:
        pstock_parsing_failed.append(new_name)
        circulation_stocks_list.append(np.nan)
        circulation_share_list.append(np.nan)
        circulation_price_list.append(np.nan)
        new_listing_stocks_list.append(np.nan)
        new_listing_share_list.append(np.nan)
        tot_price_list.append(np.nan)
        continue
        
#     try:
#         pstock_create_df()
#         failed_note()
#     except:
#         print('오류발생!')

# naver

naver_dict=dict(zip(naver_code_list,naver_name_list))

for code in naver_code_list:
    try:
        naver_search()
    except:
        s_failed_name=naver_dict.get(code)
        naver_search_failed.append(s_failed_name)
        first_price_list.append(np.nan)

        continue
    
    try:
        naver_parsing()
    except:
        p_failed_name=naver_dict.get(code)
        naver_parsing_failed.append(p_failed_name)
        first_price_list.append(np.nan)
        continue
        
update_df()
failed_note()
        
        
print(f'****** Pstock Search Failed : {pstock_search_failed}')
print(f'****** Pstock Parsing Failed: {pstock_parsing_failed}')
print(f'****** Naver Search Failed : {naver_search_failed}')
print(f'****** Naver Parsing Failed: {naver_parsing_failed}')

업데이트가 완료되었습니다.
['피플바이오', '빅히트(유가)', '넥스틴', '원방테크', '엔에이치스팩17호'] ['304840', '352820', '348210', '053080', '359090']
피플바이오
5,547,457
2020.11.16
2020.12.14
2021.01.11
빅히트
6,557,212
2020.11.12
2020.12.10
2021.01.07
넥스틴
299,812
2020.11.05
2020.12.03
2020.12.31
원방테크
1,530,069
2020.10.22
2020.11.19
2020.12.17
엔에이치스팩17호
551,120
2020.10.21
2020.11.18
2020.12.16
****** Pstock Search Failed : []
****** Pstock Parsing Failed: [['엔에이치', '17호']]
****** Naver Search Failed : []
****** Naver Parsing Failed: []
